In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None

player_pergame = pd.read_csv("../data/01_raw/Player Per Game.csv")
player_allstar = pd.read_csv("../data/01_raw/All-Star Selections.csv")
player_awards = pd.read_csv("../data/01_raw/Player Award Shares.csv")
team_stats = pd.read_csv("../data/01_raw/Team Summaries.csv")
advanced_stats = pd.read_csv("../data/01_raw/Advanced.csv")

In [ ]:
player_pergame

,seas_id,season,player_id,player,birth_year,pos,age,experience,lg,tm,...,ft_percent,orb_per_game,drb_per_game,trb_per_game,ast_per_game,stl_per_game,blk_per_game,tov_per_game,pf_per_game,pts_per_game
0,30458,2023,5025,A.J. Green,NaN,SG,23.0,1,NBA,MIL,...,1.000,0.2,1.1,1.3,0.6,0.2,0.0,0.3,0.9,4.4
1,30459,2023,5026,A.J. Lawson,NaN,SG,22.0,1,NBA,TOT,...,0.250,0.4,1.0,1.4,0.1,0.1,0.0,0.2,0.7,3.7
2,30460,2023,5026,A.J. Lawson,NaN,SG,22.0,1,NBA,MIN,...,NaN,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0
3,30461,2023,5026,A.J. Lawson,NaN,SG,22.0,1,NBA,DAL,...,0.250,0.4,1.0,1.4,0.1,0.1,0.0,0.2,0.7,3.9
4,30462,2023,4219,Aaron Gordon,NaN,PF,27.0,9,NBA,DEN,...,0.608,2.4,4.1,6.6,3.0,0.8,0.8,1.4,1.9,16.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31130,200,1947,157,Walt Miller,NaN,F,31.0,1,BAA,PIT,...,0.500,NaN,NaN,NaN,0.5,NaN,NaN,NaN,1.3,1.9
31131,201,1947,158,Warren Fenley,NaN,F,24.0,1,BAA,BOS,...,0.511,NaN,NaN,NaN,0.5,NaN,NaN,NaN,1.8,2.6
31132,202,1947,159,Wilbert Kautz,NaN,G-F,31.0,1,BAA,CHS,...,0.534,NaN,NaN,NaN,0.7,NaN,NaN,NaN,2.3,5.1
31133,203,1947,160,Woody Grimshaw,NaN,G,27.0,1,BAA,PRO,...,0.477,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.2,2.9


###Nettoyage

In [ ]:
#Adding a column to indicate whether a player was an all-star in that season.
player_pergame["all_star"] = False
star_season = []
for i in range(len(player_allstar)):
    star = player_allstar.at[i,'player']
    season = player_allstar.at[i,'season']
    star_season.append([star,season])

for i in range(len(player_pergame)):
    player = player_pergame.at[i,'player']
    season = player_pergame.at[i,'season']
    player_season = [player,season]
    if player_season in star_season:
        player_pergame.at[i,'all_star'] = True

In [ ]:
#Team success also plays a major role in selection, adding column for winning percentage.
team_stats['win_perc'] = team_stats.apply(lambda x: x['w'] / (x['w'] + x['l']), axis=1)
team_stats = team_stats[['season','abbreviation','win_perc','w','l']]
team_player_stats = pd.merge(player_pergame, team_stats,  how='left', left_on=['season','tm'], right_on = ['season','abbreviation'])

In [ ]:
# Removing all data prior to 1980. The NBA has changed drastically since its orgin and this will remove less relevant data from before the NBA-ABA merger and the three point line.
data_modern = team_player_stats[team_player_stats['season'] >= 1980]
data_modern.describe()

,seas_id,season,player_id,birth_year,age,experience,g,gs,mp_per_game,fg_per_game,...,trb_per_game,ast_per_game,stl_per_game,blk_per_game,tov_per_game,pf_per_game,pts_per_game,win_perc,w,l
count,23145.000000,23145.000000,23145.000000,621.000000,23145.000000,23145.000000,23145.000000,22472.000000,23145.000000,23145.000000,...,23145.000000,23145.000000,23145.000000,23145.000000,23145.000000,23145.000000,23145.000000,21058.000000,21058.000000,21058.000000
mean,19563.000000,2003.968157,3187.125081,1962.384863,26.688226,5.402074,47.918773,22.636525,19.771022,3.063897,...,3.466502,1.852093,0.650607,0.401158,1.217218,1.880035,8.099412,0.492991,39.580302,40.688005
std,6681.530326,12.461911,893.880944,13.215766,4.036304,3.892215,26.656619,27.988463,10.069756,2.238885,...,2.509957,1.833922,0.469532,0.490268,0.820971,0.871420,5.995677,0.153660,12.708757,12.685057
min,7991.000000,1980.000000,834.000000,1944.000000,18.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.106061,7.000000,9.000000
25%,13777.000000,1994.000000,2450.000000,1952.000000,24.000000,2.000000,24.000000,0.000000,11.400000,1.300000,...,1.600000,0.600000,0.300000,0.100000,0.600000,1.200000,3.400000,0.378049,30.000000,31.000000
50%,19563.000000,2005.000000,3186.000000,1957.000000,26.000000,4.000000,52.000000,7.000000,19.000000,2.500000,...,2.800000,1.300000,0.600000,0.200000,1.000000,1.900000,6.600000,0.500000,41.000000,40.000000
75%,25349.000000,2015.000000,3888.000000,1973.000000,29.000000,8.000000,73.000000,42.000000,28.000000,4.400000,...,4.600000,2.500000,0.900000,0.500000,1.700000,2.500000,11.500000,0.609756,49.000000,50.000000
max,31135.000000,2023.000000,5108.000000,2002.000000,44.000000,22.000000,85.000000,83.000000,44.500000,13.400000,...,18.700000,14.500000,3.700000,6.000000,5.700000,6.000000,37.100000,0.890244,73.000000,72.000000


In [ ]:
#Reformatting data so that the index is the player ID and season and dealing with missing data due to combining datasets (this is due to players playing for multiple teams in one season).
data_modern.loc[:]['season'] = pd.Categorical(data_modern['season'])
data_modern = data_modern[data_modern['win_perc'].notna()]
data_modern = data_modern[data_modern['abbreviation'].notna()]
data_modern

,seas_id,season,player_id,player,birth_year,pos,age,experience,lg,tm,...,stl_per_game,blk_per_game,tov_per_game,pf_per_game,pts_per_game,all_star,abbreviation,win_perc,w,l
0,30458,2023,5025,A.J. Green,NaN,SG,23.0,1,NBA,MIL,...,0.2,0.0,0.3,0.9,4.4,False,MIL,0.707317,58.0,24.0
2,30460,2023,5026,A.J. Lawson,NaN,SG,22.0,1,NBA,MIN,...,0.0,0.0,0.0,1.0,2.0,False,MIN,0.512195,42.0,40.0
3,30461,2023,5026,A.J. Lawson,NaN,SG,22.0,1,NBA,DAL,...,0.1,0.0,0.2,0.7,3.9,False,DAL,0.463415,38.0,44.0
4,30462,2023,4219,Aaron Gordon,NaN,PF,27.0,9,NBA,DEN,...,0.8,0.8,1.4,1.9,16.3,False,DEN,0.646341,53.0,29.0
5,30463,2023,4582,Aaron Holiday,NaN,PG,26.0,5,NBA,ATL,...,0.6,0.2,0.6,1.3,3.9,False,ATL,0.500000,41.0,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23140,8343,1980,1155,Wes Unseld,NaN,C,33.0,12,NBA,WSB,...,0.8,0.7,1.9,3.0,9.7,False,WSB,0.475610,39.0,43.0
23141,8344,1980,1897,Wiley Peck,NaN,SF,22.0,1,NBA,SAS,...,0.3,0.4,0.9,1.9,3.5,False,SAS,0.500000,41.0,41.0
23142,8345,1980,1734,Willie Smith,NaN,PG,26.0,4,NBA,CLE,...,1.2,0.0,1.5,1.8,4.8,False,CLE,0.451220,37.0,45.0
23143,8346,1980,1844,Winford Boynes,NaN,SG,22.0,2,NBA,NJN,...,0.9,0.3,1.5,2.1,8.5,False,NJN,0.414634,34.0,48.0


In [ ]:
#Adding column to indicate what conference a player is in.
east = ['MIL', 'TOT','ATL', 'IND','ORL',
       'BOS', 'DET', 'CHI', 'BRK','WAS', 'MIA','CHO','NYK',
       'CLE', 'TOR','PHI', 'CHA', 'NJN', 'CHH','WSB', 'KCK', ]
west = ['MIN', 'DAL', 'DEN','OKC','SAC', 'SAS', 'HOU', 'LAC', 'GSW',
       'POR', 'LAL','PHO', 'MEM', 'NOP','UTA','NOH','SEA','NOK', 'VAN','SDC']

data_modern['conf'] = 'east'
for index, row in data_modern.iterrows():
    team = row['tm']
    if team in west:
        data_modern.at[index,'conf'] = 'west'

f = open("../data/primary/primary.csv", "a")
f.write(data_modern.to_csv())
f.close()


###Entrainement du modèle

In [ ]:
#Creating a column to indicate the percentage of games a player played. It won't be helpful to look at total games when applying the model to 2023 teams whose data is taken from the middle of the season. Also, some seasons had less than 82 games due to lockouts/covid so that is accounted for as well.
data_modern['total_games'] = data_modern.apply(lambda x: x['w'] + x['l'], axis=1)
data_modern['games_pct'] = data_modern.apply(lambda x: x['g'] / x['total_games'], axis=1)

In [ ]:
#For this model, missing data will be filled in with the median in the column. This is only relevant for the efficency statistics (effective field goal percentage and free throw percentage). This shouldn't introduce much bias. Also removing the 2023 season as that data will be used for the final predictions.
data_model1 = data_modern.copy()
data_model1 = data_model1[data_model1['season'] != 2023]
data_model1 = data_model1.fillna(data_model1.median(numeric_only=True))

In [ ]:
#Pipeline encoding categorical variable ('pos') with OneHotEncoder and scaling numeric data with StandardScalar (sklearn).
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer

def transform(x_cols):
    return make_column_transformer(
                (OneHotEncoder(), ['pos']),
                (StandardScaler(), x_cols[1:]), # Everything except for 'pos'
    )


In [ ]:
import random

def get_train_test(data, test_size):
    global train
    global test
    train_years = list(range(1980,2023))
    test_years = []
    for i in range(test_size):
        year = random.choice(train_years)
        train_years.remove(year)
        test_years.append(year)

    train = data.loc[data['season'].isin(train_years)]
    test = data.loc[data['season'].isin(test_years)]

    print("Test Seasons:")
    print(test_years)

get_train_test(data_model1, 6)


Test Seasons:
[1989, 1995, 1988, 2018, 1984, 2012]


In [ ]:
#Fitting and scoring the logistic regression model.
x_cols = ["pos", "experience", "games_pct", "e_fg_percent", "ft_percent", "orb_per_game","drb_per_game", "trb_per_game",
         "ast_per_game", "stl_per_game", "blk_per_game", "tov_per_game", "pf_per_game","pts_per_game", "win_perc",
         "ft_per_game"]
trans = transform(x_cols)

y_col = "all_star"

model = Pipeline([
            ("tr", trans),
            ("lr", LogisticRegression(fit_intercept = False, max_iter = 1000)),
        ])

model.fit(train[x_cols], train[y_col])
model.score(test[x_cols], test[y_col])

0.9702336096458176

In [ ]:
import numpy as np

def rank_predict(test, model):
    test["prob_all_star"] = model.predict_proba(test[x_cols])[:, 1]
    test['rank'] = test.groupby(['season','conf'])['prob_all_star'].rank(ascending=False)
    test['pred_all_star'] = np.where(  (test['rank'] <= 12.0 ), True, False)

rank_predict(test, model)

In [ ]:
finalmodel = Pipeline([
            ("tr", trans),
            ("lr", LogisticRegression(fit_intercept = False, max_iter = 1000, C = 0.5455594781168515, class_weight = "balanced")),
        ])
data_modern = data_modern.fillna(data_modern.median(numeric_only=True))
train = data_modern[data_modern['season'] != 2023]
test = data_modern[data_modern['season'] == 2023]
finalmodel.fit(train[x_cols], train[y_col])
rank_predict(test, finalmodel)

In [ ]:
test['rank'] = test['rank'].astype(int)
print("2023 ALL-STAR PREDICTIONS:")
print("Eastern Conference:")
print((test[(test["pred_all_star"] == True) & (test["conf"] == 'east')].sort_values('rank')[['player','rank']]).to_string(index=False))
print()
print("Western Conference:")
print((test[(test["pred_all_star"] == True) & (test["conf"] == 'west')].sort_values('rank')[['player','rank']]).to_string(index=False))

2023 ALL-STAR PREDICTIONS:
Eastern Conference:
               player  rank
          Joel Embiid     1
Giannis Antetokounmpo     2
         Jayson Tatum     3
         Kevin Durant     4
         James Harden     5
           Trae Young     6
     Donovan Mitchell     7
         Jimmy Butler     8
         Kyrie Irving     9
         Jaylen Brown    10
        Julius Randle    11
         Jrue Holiday    12

Western Conference:
                 player  rank
           Nikola Jokić     1
           LeBron James     2
            Luka Dončić     3
          Anthony Davis     4
         Damian Lillard     5
Shai Gilgeous-Alexander     6
          Stephen Curry     7
              Ja Morant     8
           Kevin Durant     9
           Devin Booker    10
       Domantas Sabonis    11
            Paul George    12
